# DEPRECATED

This notebook is deprecated. The Hyper tuning feature is no longer necessary. The main idea of this notebook was to auto tuning the parameters of the model, but with the current implementation, the hand tests are enough to get a good score.

In [ ]:
import os
import torch
import random
from hyperopt import hp, fmin, tpe, Trials
from datetime import datetime
from artificial_dataloader import *
from pre_process import *
import torch.nn as nn
import torch.optim as optim

device = torch.device("cpu")

In [ ]:
def getData(csv_path, train_path, batch_size, num_workers=0):
    ''' get images from the folder (assets/images) and return a DataLoader object '''
    
    dataset = ArtificialLidarDatasetCNN(csv_path, train_path)

    print(f'dataset size (no augmentation): {len(dataset)}')
    #! artificial não se beneficia muito disso
    # dataset = transformData(dataset)
    print(f'dataset size (w/ augmentation): {len(dataset)}')

    train_size, val_size = int(0.8*len(dataset)), np.ceil(0.2*len(dataset)).astype('int')
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
    
    train_data = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,num_workers=num_workers)
    val_data  = DataLoader(val_dataset, batch_size=batch_size, shuffle=True,num_workers=num_workers)

    print(f'train size: {train_size}, val size: {val_size}')
    return train_data, val_data


In [ ]:

def train_model(model, criterion, optimizer, scheduler, train_loader, val_loader, num_epochs):

    train_losses = []
    val_losses = []
    predictions_list = []
    labels_list = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for i, data in enumerate(train_loader):
            images, labels = data['image'], data['labels']
            # convert to float32 and send it to the device
            # image dimension: (batch, channels, height, width)
            images = images.type(torch.float32).to(device)
            images = images.unsqueeze(1)
            # convert labels to float32 and send it to the device
            labels = [label.type(torch.float32).to(device) for label in labels]
            # convert labels to tensor
            labels = torch.stack(labels)
            # convert to format: tensor([[value1, value2, value3, value4], [value1, value2, value3, value4], ...])
            # this is: labels for each image, "batch" times -> shape: (batch, 4)
            labels = labels.permute(1, 0)    
            outputs = model(images)
            loss = criterion(outputs, labels) 
            # Loss_with_L2 = Loss_without_L2 + λ * ||w||^2
            # Calculate L2 regularization loss
            l2_regularization_loss = 0
            for param in model.parameters():
                l2_regularization_loss += torch.norm(param, 2)
            loss += weight_decay * l2_regularization_loss  # Add L2 regularization loss to the total loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()
            running_loss += loss.item()
        else:
        # valing the model
            with torch.no_grad():
                # Set the model to evaluation mode
                model.eval()
                total = 0
                val_loss = 0
                for i, data in enumerate(val_loader):
                    images, labels = data['image'], data['labels']
                    # image dimension: (batch, channels, height, width)
                    images = images.type(torch.float32).to(device)
                    images = images.unsqueeze(1)
                    labels = [label.type(torch.float32).to(device) for label in labels]
                    labels = torch.stack(labels)
                    labels = labels.permute(1, 0)
                    labels_list.append(labels)
                    total += len(labels)
                    outputs = model.forward(images) # propagação para frente
                    # get the predictions to calculate the accuracy
                    _, preds = torch.max(outputs, 1)
                    val_loss += criterion(outputs, labels).item()
                val_losses.append(val_loss/len(val_loader))
            pass
        train_losses.append(running_loss/len(train_loader))
        val_losses.append(running_loss/len(val_loader))
        print(f'[{epoch+1}/{num_epochs}] .. Train Loss: {train_losses[-1]:.5f} .. val Loss: {val_losses[-1]:.5f}')


    results = {
        'train_losses': train_losses,
        'val_losses': val_losses,
    }
    
    return results

In [ ]:
# Função objetivo para otimizar a taxa de aprendizado (lr)
def objective(lr):

    ############ HYPER PARAMS ############
    epochs = 12
    step_size = 3
    gamma = 0.5
    batch_size = 160
    weight_decay = 1e-4

    ############ DATA ############
    csv_path = "../../artificial_data/tags/Artificial_Label_Data3.csv"
    # train_path = os.getcwd() + SLASH + 'artificial_data' + SLASH + 'train4' + SLASH
    train_path = os.path.join(os.getcwd(), '..', '..', 'artificial_data', 'train3')
    train_data, val_data = getData(batch_size=batch_size, csv_path=csv_path, train_path=train_path)

    ############ MODEL ############
    model = models.resnet18(pretrained=True)
    model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(inplace=True),
        nn.Linear(512, 256),
        nn.BatchNorm1d(256),
        nn.ReLU(inplace=True),
        nn.Linear(256, 3)
    )
    model.to(device)

    criterion = nn.L1Loss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

    # Realize o treinamento do modelo
    results = train_model(model, criterion, optimizer, scheduler, train_data, val_data, epochs)

    # Retorne a métrica de desempenho que você deseja otimizar (por exemplo, perda mínima)
    # Neste exemplo, estamos considerando a última perda de validação
    return results['val_losses'][-1]

In [ ]:
# Crie o espaço de busca para a taxa de aprendizado (lr)
space = hp.uniform('lr', 1e-7, 1e-2)

# Crie um objeto Trials para rastrear os resultados da otimização
trials = Trials()

# Realize a otimização usando o TPE (Tree-structured Parzen Estimator)
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

# O melhor valor de lr será armazenado em best['lr']
best_lr = best['lr']
print("Melhor valor de lr encontrado:", best_lr)